In [1]:
# !pip install gdown
#/////////////////////////////

import gdown
import shutil
import os

# URL-адреса для скачивания файлов с Google Drive
data_zip_url = "https://drive.google.com/file/d/1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16/view?usp=sharing"
test_zip_url = "https://drive.google.com/file/d/12QrDrLT1F-X7UycvOoApXFqxTw3Zx93K/view?usp=sharing"

# Явное указание имен файлов при сохранении
data_zip_path = "/content/data.zip"
test_zip_path = "/content/test.zip"

# Загрузка и сохранение файлов под указанными именами
gdown.download(data_zip_url, data_zip_path, fuzzy=True)
gdown.download(test_zip_url, test_zip_path, fuzzy=True)

# Распаковка архивов data.zip и test.zip в текущую директорию
shutil.unpack_archive(data_zip_path, '.', 'zip')
shutil.unpack_archive(test_zip_path, '.', 'zip')

# Удаление архивов после распаковки
os.remove(data_zip_path)
os.remove(test_zip_path)

# train.csv - датасет для обучения
# sample_submission.csv - пример файла-ответа, который нужно отправить на платформу

# Сами изображения находятся по ссылкам:
# Обучение - https://drive.google.com/file/d/1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16/view?usp=sharing
# Тест - https://drive.google.com/file/d/12QrDrLT1F-X7UycvOoApXFqxTw3Zx93K/view?usp=sharing

# Поля в датасетах
# image_path - строка, являющаюся путем до изображения в случае обучения или названием изображения в случае теста
# emotion - строка, характеризующая эмоцию
# В данном задании требуется предсказать 9 базовых эмоций, таких как:
# neutral - нейтральная эмоция
# anger - гнев, злость
# contempt - презрение
# disgust - отвращение
# fear - страх
# happy - веселый
# sad - грусть
# surprise - удивленность
# uncertain - неуверенность


Downloading...
From (original): https://drive.google.com/uc?id=1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16
From (redirected): https://drive.google.com/uc?id=1TG9P5B2k3eTbC4XDxDmEc07dyAORPC16&confirm=t&uuid=945dacc8-d02e-4f87-beb1-a062664b379d
To: /content/data.zip
100%|██████████| 2.28G/2.28G [00:45<00:00, 50.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=12QrDrLT1F-X7UycvOoApXFqxTw3Zx93K
From (redirected): https://drive.google.com/uc?id=12QrDrLT1F-X7UycvOoApXFqxTw3Zx93K&confirm=t&uuid=da6900d4-8326-4b21-a8ac-b14dea677117
To: /content/test.zip
100%|██████████| 222M/222M [00:03<00:00, 59.8MB/s]


In [3]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import gdown
import shutil
from tqdm import tqdm


# маппинг эмоций
emotion_labels = {
    'anger': 0, 'contempt': 1, 'disgust': 2, 'fear': 3, 'happy': 4,
    'neutral': 5, 'sad': 6, 'surprise': 7, 'uncertain': 8
}


# обработка данных
class EmotionDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        # Заполнение списка файлов и меток
        for emotion, label in emotion_labels.items():
            emotion_dir = os.path.join(root_dir, emotion)
            for img_file in os.listdir(emotion_dir):
              # Проверка на jpg
                if img_file.endswith('.jpg'):
                    self.image_files.append(os.path.join(emotion_dir, img_file))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_files)

    # преобразовать изображение, вернуть его и метку
    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# трансформация изображений
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# создаем датасет и загрузчик
train_dataset = EmotionDataset(root_dir="/content/train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

# определяем модель
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.model = models.mobilenet_v2(pretrained=True) # предобученная модель MobileNetV2
        self.model.classifier[1] = nn.Linear(self.model.last_channel, 9)

    def forward(self, x):
        return self.model(x)

model = EmotionClassifier()
criterion = nn.CrossEntropyLoss() # функция потерь
optimizer = optim.Adam(model.parameters(), lr=0.001) # оптимизатор

# настройка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 3
# обучение
for epoch in tqdm(range(num_epochs)):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# предсказания на тестовых данных
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]

# создаем датасет и загрузчик уже для тестирования
test_dataset = TestDataset(root_dir="/content/test_kaggle", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# предсказание
model.eval()
predictions = []
with torch.no_grad():
    for images, image_files in test_loader:
        images = images.to(device, non_blocking=True)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(zip(image_files, predicted.cpu().numpy()))

# обратный маппинг меток в эмоции
reverse_emotion_labels = {v: k for k, v in emotion_labels.items()}
predictions = [(img, reverse_emotion_labels[pred]) for img, pred in predictions]

# генерация и сохранения csv файла
submission_df = pd.DataFrame(predictions, columns=['image_path', 'emotion'])
submission_df.to_csv("submission.csv", index=False)

# сохранить модельку
torch.save(model.state_dict(), "emotion_classifier.pth")

# вывод хэда файлв
print(submission_df.head())


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can

  image_path    emotion
0   1093.jpg        sad
1    304.jpg        sad
2   2077.jpg      happy
3   2816.jpg  uncertain
4    556.jpg      anger


20:40